<a href="https://colab.research.google.com/github/ivnlee/tec-mna-nlp/blob/main/semana-5/Equipo13_semana05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Continuando con caso de estudio: Amazon-Yelp-Imdb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import pandas as pd 
import numpy as np 
import re
import string
import nltk
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer

In [3]:
#nltk.download('omw-1.4')  

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### **1. Cargar datos**

In [4]:
# Cargamos los datos, observamos que para el archivo de IMDB 
# el separador es de 3 o más espacios y no un tabulador:

dfa = pd.read_csv('/content/drive/MyDrive/NLP/semana-5/amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('/content/drive/MyDrive/NLP/semana-5/imdb5.txt', sep=r'\s{3,}', names=['review','label'], header=None, encoding='utf-8', engine='python')
dfy = pd.read_csv('/content/drive/MyDrive/NLP/semana-5/yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')

# verificamos total de registros
print('Total de registros Amazon: ', dfa.shape)
print('Total de registros IMDB: ', dfi.shape)
print('Total de registros Yelp: ', dfy.shape)

Total de registros Amazon:  (1000, 2)
Total de registros IMDB:  (1000, 2)
Total de registros Yelp:  (1000, 2)


In [5]:
# unimos los 3 dataframes
df = pd.concat([dfa, dfi, dfy], ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


### **2. Limpieza y lematización**

In [6]:
def clean_tok(doc):
  # Elimina los signos de puntuación, caracteres especiales y números
  tokens = re.sub(r'[^A-Za-z\s]', ' ', doc).strip()

  # Elimina espacios en blanco adicionales y convierte a minúsculas
  tokens = ' '.join(re.findall(r'\b\w+\b', tokens.lower()))
  
  # Tokeniza por palabras, elimina stopwords y palabras de longitud menor a 1
  tokens = [w for w in tokens.split() if w not in stopwords.words('english') and len(w) > 1]

  return tokens

In [7]:
# Separamos las palabras de las etiquetas

X = df.review 
Y = df.label

In [8]:
# aplicamos la función clean_tok a cada registro de X

Xcleantok = [clean_tok(x) for x in X] 

Obsrvamos los primeros registros 

In [9]:
for x in Xcleantok[0:10]:
  print(x)

['way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversations', 'lasting', 'minutes', 'major', 'problems']
['mic', 'great']
['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume']
['several', 'dozen', 'several', 'hundred', 'contacts', 'imagine', 'fun', 'sending', 'one', 'one']
['razr', 'owner', 'must']
['needless', 'say', 'wasted', 'money']
['waste', 'money', 'time']


### Lematización

In [10]:
wnl = WordNetLemmatizer() 

def lemmatizer(doc):

  Xtmp = [wnl.lemmatize(w, 'v') for w in doc]     # lemmatiza verbos
  Xtmp = [wnl.lemmatize(w, 'a') for w in Xtmp]    # lemmatiza adjetivos
  Xtmp = [wnl.lemmatize(w, 'n') for w in Xtmp]    # lemmatiza sustantivos
  Xtmp = [wnl.lemmatize(w, 'r') for w in Xtmp]    # lemmatiza adverbios
  
  return Xtmp

In [11]:
# aplicamos la función lemmatizer a cada registro de Xcleantok

Xclean = [lemmatizer(x) for x in Xcleantok]

In [12]:
for x in Xclean[0:10]:
    print(x)

['way', 'plug', 'u', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem']
['mic', 'great']
['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume']
['several', 'dozen', 'several', 'hundred', 'contact', 'imagine', 'fun', 'send', 'one', 'one']
['razr', 'owner', 'must']
['needle', 'say', 'waste', 'money']
['waste', 'money', 'time']


### **3. Train-Validation-Test split**

In [13]:
from sklearn.model_selection import train_test_split 

x_train, x_val_test, y_train, y_val_test = train_test_split(Xclean, Y, test_size=0.30, shuffle=True, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.50, shuffle=True, random_state=42) 

# Verificamos el tamaño de los conjuntos de datos de entrenamiento, validación y prueba
print('X,y Train:', len(x_train), len(y_train)) 
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


### **4. Vocabulario**

In [14]:
from collections import Counter

In [15]:
diccionario = Counter()

# creamos diccionario con las palabras del set de entrenamiento
for k in range(len(x_train)):
  diccionario.update(x_train[k]) 

print('Tamaño del diccionario: ', len(diccionario))

Tamaño del diccionario:  3253


In [16]:
# filtramos el diccionario
frec_min = 1

dicc = {k:v for k,v in diccionario.items() if v > frec_min}

print('Tamaño del diccionario filtrado: ', len(dicc))

Tamaño del diccionario filtrado:  1429


### Filtramos los conjuntos de datos de entrenamiento, validación y prueba

In [17]:
train_x = []
for n in x_train:
    train_x.append([w for w in n if w in dicc])

val_x = []
for n in x_val:
    val_x.append([w for w in n if w in dicc])

test_x = []
for n in x_test:
    test_x.append([w for w in n if w in dicc])

### **5. Tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.**

-

### **6. Diccionario con Vectores Embebidos de FastText**

In [ ]:
#!pip install fasttext

In [1]:
import fasttext
import fasttext.util

In [2]:
# fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [18]:
ft = fasttext.load_model('cc.en.300.bin')

In [19]:
len(ft.words)

2000000

In [20]:
ft.get_dimension()

300

In [23]:
midicc = {}

# extraemos los vectores mebebidos para nuestro diccionario
for k,w in enumerate(dicc.keys()):
  tmp = ft.get_word_vector(w)
  midicc.update({w:tmp})


### Verificamos que el nuevo diccionario con vectores embebidos sea del mismo tamaño de nuestro vocabulario:

In [27]:
print(len(dicc) == len(midicc))

True


### Guardamos el diccionario en un archivo pickle

In [29]:
import pickle

#with open('midicc_emb.pickle', 'wb') as handle:
#    pickle.dump(midicc, handle, protocol=pickle.HIGHEST_PROTOCOL)


### para cargar nuevamente el archivo usamos el siguiente codigo: ###
#with open('midicc_emb.pickle', 'rb') as handle:
#    midicc = pickle.load(handle)

In [30]:
# borramos la variable ft para liberar memoria RAM
del ft 

### **7. Asignar a cada comentario filtrado el vector embebido de dimensión 300**

In [33]:
def embedding_vector(docs):
    dim = 300
    zero_vector = np.zeros(dim)
    vectors = []
    for tokens in docs:
        tmp_vect =  np.zeros(dim)
        tmp_count = 0 + 1e-5 # para evitar dividir por 0
        for token in tokens:
            if token in midicc:
                tmp_vect += midicc[token]
                tmp_count +=1
        if(tmp_count!=0):
            vectors.append(tmp_vect/tmp_count) 
        else:
            vectors.append(zero_vector)
    return vectors

In [36]:
trainEmb = embedding_vector(train_x) 

valEmb = embedding_vector(val_x)

testEmb = embedding_vector(test_x) 

In [38]:
print('X Train: ', len(trainEmb))
print('X Val: ', len(valEmb))
print('X Test: ', len(testEmb))

X Train:  2100
X Val:  450
X Test:  450


### **8. Modelos de Regresión Logística y Random Forest**

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [53]:
modeloLRemb = LogisticRegression(max_iter=50,
                                   C=1.0,
                                   solver='liblinear',
                                   random_state=42)

modeloLRemb.fit(trainEmb, y_train)


modeloRFemb = RandomForestClassifier(n_estimators=10,
                                       max_depth=4,
                                       random_state=42)

modeloRFemb.fit(trainEmb, y_train)

########################################################################################

print('LR: Train-accuracy: %.2f%%' % (100*modeloLRemb.score(trainEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloLRemb.score(valEmb, y_val)))

print('\nRF: Train-accuracy: %.2f%%' % (100*modeloRFemb.score(trainEmb, y_train)))
print('RF: Val-accuracy: %.2f%%' % (100*modeloRFemb.score(valEmb, y_val)))

LR: Train-accuracy: 80.57%
LR: Val-accuracy: 76%

RF: Train-accuracy: 80.10%
RF: Val-accuracy: 72.22%
